In [37]:
import numpy as np
import os, os.path
import matplotlib.image as matlib

In [56]:
plane = "axial"
mouse_x = 100
mouse_y = 313
axial_slice = 108
sagittal_slice = 126
coronal_slice = 137

In [57]:
npy_file = "../react-ui/src/assets/P57-16/mri_rotated/indices_" + plane + "/slice_" + str(axial_slice) + ".npy"

In [58]:
npy_slice_array = np.load(npy_file)

print(np.isfortran(npy_slice_array))
print(npy_slice_array.shape)
#print(npy_slice_array[45])

False
(450, 226)


In [59]:
# get the block number from the numpy array
block = npy_slice_array[mouse_y][mouse_x]
print(block)

35


In [60]:
# define a vector to be used later (to get the histology coords)
# an earlier version of this notebook did +1 to the mri_slice here, need to check if this is necessary.
# need to find out the correct order of params when creating the vector (depending on the current plane)
if block == 0:
    print("block number is 0, no further calculations are necessary as there is no associated histology")

    
#if plane == "sagittal":
#  mri_vector=np.array([float(mri_slice), float(mouse_y), float(mouse_x), 1.0])
#elif plane == "coronal":
#  mri_vector=np.array([float(mri_slice), float(mouse_y), float(mouse_x), 1.0])
#elif plane == "axial":
#  mri_vector=np.array([float(mouse_x), float(mouse_y), float(mri_slice), 1.0])

mri_vector = np.array([float(sagittal_slice), float(coronal_slice), float(axial_slice), 1.0])

In [61]:
# open the matrix txt file
matrix_txt = open("../react-ui/src/assets/P57-16/mri/matrices/block_" + str(block) + ".txt")
read_matrix_txt = matrix_txt.read()

In [62]:
# parse the text file and convert it to an array
matrix = read_matrix_txt.replace('\n', ' ')
matrix = matrix.split(" ")
matrix = matrix[:-1] # pops the last element in the array

In [63]:
# convert the array of strings to an array of floats
matrix_as_floats = []
for item in matrix:
    matrix_as_floats.append(float(item))

In [64]:
# divide matrix array into chuncks to form a multidimensional array
def divide_chunks(l, n):
    for i in range(0, len(l), n): 
        yield l[i:i + n]

# how many elements should have
n = 4
  
chunked_array = list(divide_chunks(matrix_as_floats, n))
print(chunked_array)

[[0.4074435428402476, -3.6811013654388773, 1.5910461299389174, 641.7138519021559], [-0.5385821866938263, 1.2344138477219517, 3.797578063900766, -237.01869453446682], [-0.5813015221665628, -0.09033068469291156, -0.07897139171600423, 94.13156309649324], [0.0, 0.0, 0.0, 1.0]]


In [65]:
transform = np.array(chunked_array)
print(transform)

[[ 4.07443543e-01 -3.68110137e+00  1.59104613e+00  6.41713852e+02]
 [-5.38582187e-01  1.23441385e+00  3.79757806e+00 -2.37018695e+02]
 [-5.81301522e-01 -9.03306847e-02 -7.89713917e-02  9.41315631e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [66]:
blockvector = transform.dot(mri_vector)

In [67]:
print(blockvector)

[ 3.60573833e+02  2.74373078e+02 -1.66428048e-02  1.00000000e+00]


In [ ]:
# test_transform = np.array([[0.31385434307807203, -3.664457652247729, 1.6199030926177134, 666.0080271821023],
# [-0.7744411837137424, 1.4870493065731998, 3.6535143658756977, -139.31333561581766],
# [-1.2110952755381341, -0.18593341100766062, -0.1914204859080043, 118.27995513465419],
# [0.0, 0.0, 0.0, 1.0]])

In [2]:
histology = "../react-ui/src/assets/P57-16/histology"


In [91]:
folder = os.path.join(histology,"45")
slice_folder = os.path.join(folder,"slices_HE")
if os.path.exists(slice_folder):
    print(slice_folder)
else:
    print("the folder does not exist")
images = [name for name in os.listdir(slice_folder) if os.path.isfile(os.path.join(slice_folder, name))]
print(len(images))
image = None
if len(images) > 0:
    image = images[0]

../react-ui/src/assets/P57-16/histology/45/slices_HE
39


In [92]:
if not image == None:
    fullPath = os.path.join(slice_folder, image)
    readImage = matlib.imread(fullPath)
    print(readImage.shape)

(841, 592, 3)
